In [84]:
import numpy as np

There is a question - can we use a dumb laser rangefinder (or something similar) to do fish length detection?

Let's try to answer this with a laser pointer.  This solution uses a single laser pointers to project a dot onto a surface that can then be imaged.  Due to perspective, the corresponding pixel location of the laser dot changes with the distance to the surface, so we can estimate the distance of the surface.

Let's set the stage for this.  For all simulations, we will use a pinhole camera model.  We'll make life easy and assume an Olympus TG-6 at the widest angle setting, and ignore the air-lens-water interface.

Thus, we have the following parameters:
- Sensor size (width x height): 4000 x 3000
- Pixel Pitch (um): 1.5
- Focal Length (mm): 4.5

Let's place one lasers pointed parallel to the optical axis with a baseline separation of 50 mm to the left of the optical axis.  We will define the global origin on center of the sensor, with $x$ along the width of the sensor, $y$ along the height of the sensor, and $z$ pointing out of the camera into the lens.

In [85]:
sensor_size_px = np.array([4000, 3000])
pixel_pitch_mm = 0.0015
focal_length_mm = 4.5
laser_1_origin = np.array([0.05, 0.05, 0])
laser_1_axis = np.array([0, 0, 1])

Let's assume we have a flat object 1 m from the sensor.  The lasers will project one dot onto the object.  Specifically, we will have a dot at [-0.050, 0, 1].

In [86]:
plane_normal = np.array([0, 0, 1])
plane_origin = np.array([0, 0, 5])

For a plane defined as $\left<\left(\mathbf{p} - \mathbf{p_0}\right), \mathbf{n}\right> = 0$ and a vector defined as $\mathbf{p} = \mathbf{l_0} + \mathbf{l}d, d \in \mathbb{R}$, the point of intersection is the point on the vector such that $d = \frac{\left<\left(\mathbf{p_0} - \mathbf{l_0}\right), \mathbf{n}\right>}{\left<\mathbf{l}, \mathbf{n}\right>}$

In [87]:
laser_1_scalar = np.dot((plane_origin - laser_1_origin), plane_normal) / np.dot(laser_1_axis, plane_normal)
laser_1_dot = laser_1_origin + laser_1_axis * laser_1_scalar
laser_1_dot

array([0.05, 0.05, 5.  ])

Any point $[x_1, x_2, x_3]$ in the world in the field of view will project onto the image plane as follows:
$$ \begin{pmatrix}y_1\\y_2\end{pmatrix} = -\frac{f}{x_3}\begin{pmatrix}x_1\\x_2\end{pmatrix} $$

In [88]:
laser_1_projection = -focal_length_mm / 1e3 / laser_1_dot[2] * laser_1_dot[0:2] / (pixel_pitch_mm / 1e3) # in pixels
laser_1_projection

array([-30., -30.])

In [89]:
def image_coordinate_to_projected_point(S):
    assert isinstance(S, np.ndarray)
    # S += sensor_size_px / 2
    I_project = S * pixel_pitch_mm / 1e3
    I = np.array([I_project[0], I_project[1], -focal_length_mm / 1e3])
    return I


In [90]:
projected_point = image_coordinate_to_projected_point(laser_1_projection)
projected_point

array([-4.5e-05, -4.5e-05, -4.5e-03])

In [91]:
final_laser_axis = -projected_point / np.linalg.norm(projected_point)
final_laser_axis

array([0.009999  , 0.009999  , 0.99990001])

Let $\ell$ be the laser origin, $\alpha$ be the laser axis, $v$ be the final laser axis, $p$ be the real world laser point

Because we know both are the laser line, they should intersect.

Thus, we can define the following constants:

$$k_1 | k_1 \alpha + \ell = p$$
$$k_2 | k_2 v = p$$

Thus, we can break this down into three equations with two unknowns.

$$k_1 \alpha + \ell = k_2 v$$

Thus:
$$ k_1 a_x + \ell_x = k_2 v_x$$
$$ k_1 a_z = k_2 v_z$$
$$ k_1 = \frac{k_2 v_z}{a_z}$$
$$a_x \left(\frac{k_2 v_z}{a_z}\right) + \ell_x = k_2 v_x$$
$$\left(\frac{a_x v_z}{a_z}\right) k_2 - k_2 v_x = -\ell_x$$
$$k_2 \left(\frac{a_x v_z}{a_z} - v_x\right) = -\ell_x$$
$$k_2 = \frac{-\ell_x}{\frac{a_x v_z}{a_z} - v_x} = \frac{-a_z \ell_x}{a_x v_z - a_z v_x}$$

In [92]:
k_2 = (-laser_1_axis[2] * laser_1_origin[0]) / (laser_1_axis[0] * final_laser_axis[2] - laser_1_axis[2] * final_laser_axis[0])
k_2

5.0004999750025005

In [93]:
world_point = k_2 * final_laser_axis
world_point

array([0.05, 0.05, 5.  ])

In [94]:
laser_1_origin

array([0.05, 0.05, 0.  ])

$$\begin{bmatrix}
\alpha_x & -v_x\\
\alpha_y & -v_y\\
\alpha_z & -v_z
\end{bmatrix} \begin{bmatrix}
k_1\\
k_2
\end{bmatrix} = \begin{bmatrix}
- \ell_x\\
- \ell_y\\
0
\end{bmatrix}$$

$$A^T A = \begin{bmatrix}
\alpha_x & \alpha_y & \alpha_z\\
-v_x & -v_y & -v_z
\end{bmatrix} \begin{bmatrix}
\alpha_x & -v_x\\
\alpha_y & -v_y\\
\alpha_z & -v_z
\end{bmatrix} = \begin{bmatrix}
\alpha_x^2 + \alpha_y^2 + \alpha_z^2 & -\alpha_x v_x - \alpha_y v_y - \alpha_z v_y\\
-\alpha_x v_x - \alpha_y v_y - \alpha_z v_y & v_x^2 + v_y^2 + v_z^2
\end{bmatrix} = \begin{bmatrix}
||\alpha||^2 & -\alpha^T v\\
-\alpha^T v & ||v||^2
\end{bmatrix}$$

Because $\alpha$ and $v$ are unit vectors

$$A^T A = \begin{bmatrix}
1 & -\alpha^T v\\
-\alpha^T v & 1
\end{bmatrix}$$

$$A^T \ell = \begin{bmatrix}
\alpha_x & \alpha_y & \alpha_z\\
-v_x & -v_y & -v_z
\end{bmatrix} \begin{bmatrix}
- \ell_x\\
- \ell_y\\
0
\end{bmatrix} = \begin{bmatrix}
- \ell_x \alpha_x + -\ell_y \alpha_y\\
\ell_x v_x + \ell_y v_y
\end{bmatrix} = \begin{bmatrix}
-\alpha^T \ell\\
v^T \ell
\end{bmatrix}$$

$$\begin{bmatrix}
1 & -\alpha^T v & -\alpha^T \ell\\
-\alpha^T v & 1 & v^T \ell
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
-1 & \frac{1}{\alpha^T v} & \frac{v^T \ell}{\alpha^T v}
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
0 & \frac{1}{\alpha^T v} - \alpha^T v & -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v}
\end{bmatrix}$$

$$\sim \begin{bmatrix}
1 & -\alpha^T v & -\alpha^T \ell\\
0 & \frac{1}{\alpha^T v} - \alpha^T v & -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v}
\end{bmatrix} \sim \begin{bmatrix}
1 & - \alpha^T v & -\alpha^T \ell\\
0 & 1 & \left( -\alpha^T \ell + \frac{v^T \ell}{\alpha^T v} \right) \left( \frac{1}{\frac{1}{\alpha^T v}} - \alpha^T v \right)
\end{bmatrix}$$

$$k_2 = \left( -\alpha \ell + \frac{v^T \ell}{\alpha^T v} \right) \left( \frac{1}{\frac{1}{\alpha^T v} - \alpha^T v} \right)$$
$$ = \frac{-\alpha^T \ell \alpha^T v + v^T \ell}{\alpha^T v} \frac{\alpha^T v}{1 - \alpha^T v \alpha^T v}$$
$$ = \frac{-\alpha^T \ell \alpha^T v + v^T \ell}{1 - (\alpha^T v)^2}$$

In [95]:

k2_ls = ((final_laser_axis.T @ laser_1_origin) - ((laser_1_axis.T @ laser_1_origin) * (laser_1_axis.T @ final_laser_axis))) / \
                    (1 - (laser_1_axis.T @ final_laser_axis) ** 2)

k2_ls

5.000499975003039

In [96]:
world_point = k2_ls * final_laser_axis
world_point

array([0.05, 0.05, 5.  ])

In [97]:
laser_1_origin

array([0.05, 0.05, 0.  ])

### Calibration

In [98]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cup")

In [99]:
def dot(A: torch.Tensor, B: torch.Tensor):
    return torch.bmm(A.unsqueeze(dim=1), B.unsqueeze(dim=2)).squeeze()

In [100]:
sensor_size_px = torch.tensor([4000, 3000], device=device, dtype=torch.float32)
pixel_pitch_mm = 0.0015
focal_length_mm = 4.5
laser_origin = torch.tensor([0.05, 0.05, 0], device=device, dtype=torch.float32)
laser_axis = torch.tensor([0, 0, 1], device=device, dtype=torch.float32)

In [101]:
plane_normal = torch.tensor([0, 0, 1], device=device, dtype=torch.float32)

plane_origins = torch.zeros((49, 3), device=device, dtype=torch.float32)
plane_origins[:, 2] = torch.linspace(20, 500, 49, device=device, dtype=torch.float32) / 100

In [102]:
laser_scalars = dot((plane_origins - laser_origin.repeat((49, 1))), plane_normal.repeat((49, 1))) / torch.dot(laser_axis, plane_normal)
laser_dots = laser_origin.repeat((49, 1)) + laser_axis.repeat((49, 1)) * laser_scalars.unsqueeze(dim=1)
laser_dots

tensor([[0.0500, 0.0500, 0.2000],
        [0.0500, 0.0500, 0.3000],
        [0.0500, 0.0500, 0.4000],
        [0.0500, 0.0500, 0.5000],
        [0.0500, 0.0500, 0.6000],
        [0.0500, 0.0500, 0.7000],
        [0.0500, 0.0500, 0.8000],
        [0.0500, 0.0500, 0.9000],
        [0.0500, 0.0500, 1.0000],
        [0.0500, 0.0500, 1.1000],
        [0.0500, 0.0500, 1.2000],
        [0.0500, 0.0500, 1.3000],
        [0.0500, 0.0500, 1.4000],
        [0.0500, 0.0500, 1.5000],
        [0.0500, 0.0500, 1.6000],
        [0.0500, 0.0500, 1.7000],
        [0.0500, 0.0500, 1.8000],
        [0.0500, 0.0500, 1.9000],
        [0.0500, 0.0500, 2.0000],
        [0.0500, 0.0500, 2.1000],
        [0.0500, 0.0500, 2.2000],
        [0.0500, 0.0500, 2.3000],
        [0.0500, 0.0500, 2.4000],
        [0.0500, 0.0500, 2.5000],
        [0.0500, 0.0500, 2.6000],
        [0.0500, 0.0500, 2.7000],
        [0.0500, 0.0500, 2.8000],
        [0.0500, 0.0500, 2.9000],
        [0.0500, 0.0500, 3.0000],
        [0.050

In [103]:
laser_projections = -focal_length_mm / 1e3 / laser_dots[:, 2].unsqueeze(dim=1) * laser_dots[:, 0:2] / (pixel_pitch_mm / 1e3) # in pixels
laser_projections

tensor([[-750.0000, -750.0000],
        [-500.0000, -500.0000],
        [-375.0000, -375.0000],
        [-300.0000, -300.0000],
        [-250.0000, -250.0000],
        [-214.2857, -214.2857],
        [-187.5000, -187.5000],
        [-166.6667, -166.6667],
        [-150.0000, -150.0000],
        [-136.3636, -136.3636],
        [-125.0000, -125.0000],
        [-115.3846, -115.3846],
        [-107.1429, -107.1429],
        [-100.0000, -100.0000],
        [ -93.7500,  -93.7500],
        [ -88.2353,  -88.2353],
        [ -83.3333,  -83.3333],
        [ -78.9474,  -78.9474],
        [ -75.0000,  -75.0000],
        [ -71.4286,  -71.4286],
        [ -68.1818,  -68.1818],
        [ -65.2174,  -65.2174],
        [ -62.5000,  -62.5000],
        [ -60.0000,  -60.0000],
        [ -57.6923,  -57.6923],
        [ -55.5555,  -55.5555],
        [ -53.5714,  -53.5714],
        [ -51.7241,  -51.7241],
        [ -50.0000,  -50.0000],
        [ -48.3871,  -48.3871],
        [ -46.8750,  -46.8750],
        

In [104]:
def image_coordinate_to_projected_point(
    image_point: torch.Tensor, pixel_pitch_mm: float, focal_length_mm: float, device: torch.device
):
    count, _ = image_point.shape

    I_project = image_point * pixel_pitch_mm / 1e3
    I = torch.zeros((count, 3), device=device)
    I[:, :2] = I_project
    I[:, 2] = -focal_length_mm / 1e3

    return I

In [108]:
def laser2world(coord: torch.Tensor, laser_origin: torch.Tensor, laser_axis: torch.Tensor):
    count, _ = coord.shape

    laser_axises: torch.Tensor = laser_axis.repeat((count, 1))
    laser_origins = laser_origin.repeat((count, 1))

    projected_points = image_coordinate_to_projected_point(
        coord, pixel_pitch_mm, focal_length_mm, device
    )
    final_laser_axis = (
        -1
        * projected_points
        / torch.linalg.norm(projected_points, axis=1).unsqueeze(dim=1)
    )

    point_constants = (
        dot(final_laser_axis, laser_origins)
        - (
            dot(laser_axises, laser_origins)
            * dot(laser_axises, final_laser_axis)
        )
    ) / (1 - dot(laser_axises, final_laser_axis) ** 2)

    return point_constants.unsqueeze(dim=1) * final_laser_axis

In [156]:
grad_laser_axis = torch.rand((1, 3), device=device, requires_grad=True)
grad_laser_origin = torch.rand((1, 3), device=device, requires_grad=True)

# grad_laser_axis = torch.tensor(laser_axis, device=device, requires_grad=True)
# grad_laser_origin = torch.tensor(laser_origin, device=device, requires_grad=True)

mse = torch.nn.MSELoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(
    [grad_laser_axis, grad_laser_origin], lr=learning_rate
)

i = 0
while True:
    calculated = laser2world(laser_projections, grad_laser_origin, grad_laser_axis)

    error = mse(calculated, laser_dots)

    if error.item() <= 0.0001:
        break

    if i % 100 == 0:
        print(
            f"Error: {error.item()}, Origin: {grad_laser_origin.cpu().detach().numpy()}, Axis: {grad_laser_axis.cpu().detach().numpy()}"
        )

    optimizer.zero_grad()

    error += torch.abs(grad_laser_origin[0, 2])
    error += torch.abs(torch.linalg.norm(grad_laser_axis) - 1)
    error.backward()

    optimizer.step()

    i += 1

Error: 1.5792639255523682, Origin: [[0.08180892 0.0372045  0.9466525 ]], Axis: [[0.26190114 0.01231116 0.13043831]]
Error: 1.5683594942092896, Origin: [[0.07154489 0.04679319 0.95654243]], Axis: [[0.2718864  0.02335639 0.14049424]]
Error: 1.55782949924469, Origin: [[0.06059732 0.05470376 0.96612114]], Axis: [[0.2818275  0.0364664  0.15068997]]
Error: 1.5476595163345337, Origin: [[0.04904961 0.0596094  0.97537935]], Axis: [[0.29171875 0.05060035 0.16100124]]
Error: 1.5377986431121826, Origin: [[0.03699707 0.05941441 0.98431766]], Axis: [[0.30155867 0.0651528  0.17140733]]
Error: 1.5281568765640259, Origin: [[0.02452272 0.05176767 0.99293613]], Axis: [[0.31134886 0.07984657 0.18189958]]
Error: 1.5186277627944946, Origin: [[0.01169723 0.0370548  1.0012311 ]], Axis: [[0.32109332 0.09456218 0.19248068]]
Error: 1.509154200553894, Origin: [[-0.00142033  0.01876608  1.0091882 ]], Axis: [[0.33079746 0.10922711 0.20315765]]
Error: 1.4997292757034302, Origin: [[-1.4780693e-02 -4.8270272e-04  1.01

In [157]:
grad_laser_axis

tensor([[-0.0285, -0.0552,  0.9998]], device='cuda:0', requires_grad=True)

In [143]:
laser_axis

tensor([0., 0., 1.], device='cuda:0')

In [158]:
grad_laser_origin

tensor([[-3.1989e-01,  3.5886e-01, -5.2117e-06]], device='cuda:0',
       requires_grad=True)

In [145]:
laser_origin

tensor([0.0500, 0.0500, 0.0000], device='cuda:0')

In [159]:
laser2world(laser_projections, grad_laser_origin, grad_laser_axis)

tensor([[0.0303, 0.0303, 0.1212],
        [0.0340, 0.0340, 0.2037],
        [0.0368, 0.0368, 0.2947],
        [0.0391, 0.0391, 0.3915],
        [0.0410, 0.0410, 0.4923],
        [0.0426, 0.0426, 0.5962],
        [0.0439, 0.0439, 0.7020],
        [0.0450, 0.0450, 0.8094],
        [0.0459, 0.0459, 0.9177],
        [0.0467, 0.0467, 1.0267],
        [0.0473, 0.0473, 1.1360],
        [0.0479, 0.0479, 1.2455],
        [0.0484, 0.0484, 1.3550],
        [0.0488, 0.0488, 1.4643],
        [0.0492, 0.0492, 1.5734],
        [0.0495, 0.0495, 1.6822],
        [0.0497, 0.0497, 1.7906],
        [0.0500, 0.0500, 1.8985],
        [0.0501, 0.0501, 2.0059],
        [0.0503, 0.0503, 2.1127],
        [0.0504, 0.0504, 2.2191],
        [0.0505, 0.0505, 2.3247],
        [0.0506, 0.0506, 2.4298],
        [0.0507, 0.0507, 2.5342],
        [0.0507, 0.0507, 2.6380],
        [0.0508, 0.0508, 2.7409],
        [0.0508, 0.0508, 2.8434],
        [0.0508, 0.0508, 2.9453],
        [0.0508, 0.0508, 3.0463],
        [0.050

In [160]:
laser_dots

tensor([[0.0500, 0.0500, 0.2000],
        [0.0500, 0.0500, 0.3000],
        [0.0500, 0.0500, 0.4000],
        [0.0500, 0.0500, 0.5000],
        [0.0500, 0.0500, 0.6000],
        [0.0500, 0.0500, 0.7000],
        [0.0500, 0.0500, 0.8000],
        [0.0500, 0.0500, 0.9000],
        [0.0500, 0.0500, 1.0000],
        [0.0500, 0.0500, 1.1000],
        [0.0500, 0.0500, 1.2000],
        [0.0500, 0.0500, 1.3000],
        [0.0500, 0.0500, 1.4000],
        [0.0500, 0.0500, 1.5000],
        [0.0500, 0.0500, 1.6000],
        [0.0500, 0.0500, 1.7000],
        [0.0500, 0.0500, 1.8000],
        [0.0500, 0.0500, 1.9000],
        [0.0500, 0.0500, 2.0000],
        [0.0500, 0.0500, 2.1000],
        [0.0500, 0.0500, 2.2000],
        [0.0500, 0.0500, 2.3000],
        [0.0500, 0.0500, 2.4000],
        [0.0500, 0.0500, 2.5000],
        [0.0500, 0.0500, 2.6000],
        [0.0500, 0.0500, 2.7000],
        [0.0500, 0.0500, 2.8000],
        [0.0500, 0.0500, 2.9000],
        [0.0500, 0.0500, 3.0000],
        [0.050